In [1]:
try:
    sc and spark
except NameError as e:
    import pyspark
    import pyspark.sql
    
    sc = pyspark.SparkContext()
    spark = pyspark.sql.SparkSession.builder.appName("test").getOrCreate()

In [2]:
import graphframes as gf
records = spark.read.option("header","true").csv("data/small_test_data.csv")

In [3]:
records.show()

+--------------+-----------------+--------------------+----------+----------------+-----------+---------+
|            ip|              mac|            hostname| serial_no|           owner|source_name|record_id|
+--------------+-----------------+--------------------+----------+----------------+-----------+---------+
|          null|29:0b:58:3d:26:6d|isoulsby3v@behanc...|KXHGN8D8KM|    Izzy Soulsby|         LD|        0|
|          null|a9:11:e3:35:23:fa| cpollak74@nifty.com|Y5K30C65M3| Cathlene Pollak|         LD|        1|
|          null|90:22:03:25:11:d8|  ckissock53@php.net|7ZGA85QA5H|   Colin Kissock|         LD|        2|
|          null|4c:31:07:e9:9a:90|  bdruhan1m@1688.com|U0UOS9R43J|Brittaney Druhan|         LD|        3|
|106.60.189.138|             null|  bdruhan1m@1688.com|      null|            null|         SY|        4|
|106.60.189.138|4c:31:07:e9:9a:90|                null|      null|            null|         DT|        5|
+--------------+-----------------+------------

In [ ]:
# generate edges (self-join)
# generate vertices
# create graph
# run connected components